<a href="https://colab.research.google.com/github/epogrebnyak/rides/blob/master/rides2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание работ 

Цель - показать сходимость треков поездок с разными параметрами и допущениями.

Требования: нам нужна более или менее автономная процедура с минимумом ручных действий, которая показывает близкие треки.

Шаги решения:

- выбираем группы рейсов для анализа
и период анализа
- задаем параметры алгоритма сходимости
- запускаем вычисления
- просматриваем примеры близких рейсов

Выбор групп рейсов включает в себя:

- типы автомобилей (автобусы, грузовые, легковые, спецтранспорт)
- за какое время или какие дни берем рейсы
- другие признаки рейсов

На выходе:

- ранжирование всех треков попарно по сходимости, `n*(n-1)/2` пар
- каталог наиболее близких маршрутов (примеров сходимости)

Отличия от "тяжелого" алгоритма:

- в данном алгоритме мы смотрим на сходимость треков по их географии, и не проверяем сходимость по времени поездки (в отличие от "тяжелого" алгоритма, где мы смотрим конкретное время поездки последовательно от начала поездки)
- время поездки может произвольно переноситься (напрямую применимо к грузовым, менее применимо  к пассажирским и легковым автомобилям)
- мы не учитываем отдельно остановки, рассматриваем близость произвольных всех маршрутов 
- нет привяки к конкретным городам



Гипотезы и определения:

1. треки разбиваются по суткам, все заказы внутри суток объединяются
1. первая точка каждой зарегистрированной поездки - остановка с нулевой продолжительностью
1. "автобусы" - автомобили вместимостью не менее 8 человек


# Подготовка данных

## Устанавливаем скрипты для работы с данными


Скрипты копируются из удаленного репозитария путем копирования `zip` файла.

Альтернативы доставки:

- пакет для `git clone && pip install`
- держать скрипты в папке Dropbox
- копировать средствами `google.colab`
- копировать [по одному файлу](https://github.com/epogrebnyak/rides/issues/1)

Критерии выбора способы доставки: 
- простота путей к файлам данных
- плоская структура файлов
- контроль версий и тестирование скриптов
- чистота структуры папки.

## Определяем источник данных






Для доступа к данным необходимо предоставить URL к zip-файлу (устанавливается в следующей ячейке в константе `RAW_DATA_URL`).

Чтобы обычная ссылка Dropbox стала скачиваемой напрямую в ней надо заменить адрес `www.dropbox.com` на `dl.dropboxusercontent.com`.

## Загружаем исходные треки поездок


In [1]:
import os

# Устанавливаем пакеты
if not os.path.exists('master.zip'):
  !wget -O master.zip https://github.com/epogrebnyak/rides/archive/master.zip
  !unzip -o master.zip -d .
for file in ['curl.py', 'distance.py', 'rides.py', 'vectors.py']:
   if not os.path.exists(file):
     !cp -v ./rides-master/{file} .

# Ссылка URL c данными и имя файла с данными
RAW_DATA_URL = ("https://dl.dropboxusercontent.com" 
                "/sh/hibzl6fkzukltk9/AABTFmhvDvxyQdUaBsKl4h59a/"
                "data_samples_json.zip")

# Загружаем файлы программ
from curl import Files
if not os.path.exists(Files.RAW_ZIP_FILE):
  !wget -O {Files.RAW_ZIP_FILE} {RAW_DATA_URL}
  !mkdir -p {Files.RAW_JSON_FOLDER}
  !unzip -o {Files.RAW_ZIP_FILE}  -d {Files.RAW_JSON_FOLDER} > /dev/null

# Проверим наличие файлов данных
!echo Raw file count:
!find {Files.RAW_JSON_FOLDER} -type f | wc -l  

--2020-03-30 19:38:56--  https://github.com/epogrebnyak/rides/archive/master.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/epogrebnyak/rides/zip/master [following]
--2020-03-30 19:38:56--  https://codeload.github.com/epogrebnyak/rides/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [ <=>                ] 638.72K  --.-KB/s    in 0.04s   

2020-03-30 19:38:57 (15.3 MB/s) - ‘master.zip’ saved [654047]

Archive:  master.zip
5de8d7a3bd66f1c77f1666b22b708557a0329dc3
   creating: ./rides-master/
  inflating: ./rides-master/.gitignore  
  inflating: ./rides-master/README.md 

## Создаем локальные CSV файлы и считываем данные в датафреймы 

- Получаем константы `VEHICLES` и `VEHICLE_TYPES`.
- Создаем переменную `df_full`. 
- Переменную также можно проверить на корректную длину, например, ```assert df_full.shape == (9065205, 9)```. Проверка сейчас убрана из кода, потому что заранее размер фрейма для новых данных будет неизвестен. 
- Время работы ячейки ниже 2-5 минут, но при перезапуске - сохраняется.


In [0]:
from rides import Dataset

# Привязываемся к местным каталогам
d = Dataset(raw_data_dir=Files.RAW_JSON_FOLDER,
            cache_dir=Files.TEMP_FOLDER)

# Строим промежуточные файлы - обрабатывается 9 млн строк
d.build()

# Избегаем повторного создания переменной df_full, если она уже в памяти
#
# Eсли надо пересобрать датасет, то можно перезапустить colab или
# del df_full 
#
try:
  df_full
except NameError:  
  df_full = d.get_dataframe()

# Получаем константы
VEHICLES = d.vehicle_summary()
VEHICLE_TYPES = d.vehicle_types()

# Показываем пример данных
df_full.head()

9065205 rows [02:50, 53188.27 rows/s]


dataframe_with_distances() took 182.60 sec


Для многих машин типа "freight" указана грузоподъемность 1.

In [0]:
def category(car_id, mapper=VEHICLES):
  return mapper[mapper.car_id == car_id].category.iloc[0] 

assert category('1402ab8f-478a-4d9b-bc78-b5f2ba4db319') == 'Автобус\Городской'
VEHICLES.query('type=="freight"').cat_carry_weight.sort_values()

# Поиск сходимости


Приницпиальный алгоритм для пары треков:

1. надо пометить все точки трека, которые были рядом с любыми точками другого трека, вне зависимости от времени. 

2. определить, были ли эти близкие точки реалистичными по времени (могла ли произойти доставка). 


Проблемы:

- нужно посчитать около 3000*3000 дистанций, если мы ведем перебор по всем точкам пути
- не ясно, что считать хорошим пересечением или реалистичной доставкой. Например, в случае если машины паркуются на одной базе, но ездят в разные стороны, их треки сходятся утром и вечером, но это не делает поездки взаимозаменяемыми.

Какие задачи нам надо решить дальше:

- создание выборки треков
- отрисовка выборки или пары треков
- упрощение трека, для того чтобы уменьшить количество расчетов
- собственно поиск сходимости 



### Функции работы с треком

In [0]:
"""Функции работы с одним треком"""

from typing import Tuple
import numpy as np
import pandas as pd 

from distance import distance_km

Trip = pd.DataFrame 
Coord = Tuple[float, float]

def lat_lon(x: pd.Series) -> Coord:
  """Получить широту и долготу точки трека."""
  return (x.lat, x.lon)

def rev(coord: Coord) -> Coord:
  """Переставить местами широту и долготу (для отрисовки на графиках)"""
  return coord[1], coord[0]

def fst(t: Trip):
  return lat_lon(t.iloc[0])

def lst(t: Trip):
  return lat_lon(t.iloc[-1])

def drift(f):
  return distance_km(fst(t), lst(t)) 

def is_same_base(f, search_radius_km=1):
  return drift(t) < search_radius_km

def ends(t):
  a = fst(t)
  b = lst(t)
  return rev(a), rev(b) # (x1, y1), (x2, y2)

def points(t) -> np.ndarray:
  tuples = t.apply(lat_lon, axis=1).to_list()
  return np.array(tuples)

def length(t, precision=1):
  return round(t.iloc[-1].milage, precision) 

def inner(t, a=0.15, b=0.85) -> [bool]:
   x = t.milage / length(t)
   return (x >= a) & (x <= b)

def car_id(t):
   return t.iloc[0].car

def date(t):
   return t.iloc[0].date.strftime('%Y-%m-%d')

from dataclasses import dataclass

@dataclass
class TripSummary:
  milage: float
  date: str
  category: str
  car: str

def describe(t: Trip) -> TripSummary:
  c = car_id(t)
  return TripSummary(length(t), date(t), category(c), c)

In [0]:
import pandas as pd
t1 = pd.read_json('{"time":{"414822":"06:48:12","415179":"09:02:23","415503":"09:47:11","415631":"10:09:45","415895":"10:37:44","416077":"10:55:31","416293":"11:30:33","416930":"13:21:53","417115":"13:38:10","417251":"13:53:37","417298":"13:59:48"},"lon":{"414822":20.492528,"415179":20.58852,"415503":20.615398,"415631":20.543102,"415895":20.493774,"416077":20.560882,"416293":20.47951,"416930":20.523266,"417115":20.520946,"417251":20.56599,"417298":20.618018},"lat":{"414822":54.693928,"415179":54.682088,"415503":54.678524,"415631":54.667,"415895":54.696848,"416077":54.711504,"416293":54.692384,"416930":54.688972,"417115":54.67614,"417251":54.644876,"417298":54.677288},"date":{"414822":1568246400000,"415179":1568246400000,"415503":1568246400000,"415631":1568246400000,"415895":1568246400000,"416077":1568246400000,"416293":1568246400000,"416930":1568246400000,"417115":1568246400000,"417251":1568246400000,"417298":1568246400000},"car":{"414822":"a8185e3b-82a6-42a6-8673-4b3ca8349b7b","415179":"a8185e3b-82a6-42a6-8673-4b3ca8349b7b","415503":"a8185e3b-82a6-42a6-8673-4b3ca8349b7b","415631":"a8185e3b-82a6-42a6-8673-4b3ca8349b7b","415895":"a8185e3b-82a6-42a6-8673-4b3ca8349b7b","416077":"a8185e3b-82a6-42a6-8673-4b3ca8349b7b","416293":"a8185e3b-82a6-42a6-8673-4b3ca8349b7b","416930":"a8185e3b-82a6-42a6-8673-4b3ca8349b7b","417115":"a8185e3b-82a6-42a6-8673-4b3ca8349b7b","417251":"a8185e3b-82a6-42a6-8673-4b3ca8349b7b","417298":"a8185e3b-82a6-42a6-8673-4b3ca8349b7b"},"milage":{"414822":0.0,"415179":7.9059718093,"415503":15.9581903169,"415631":23.9072036886,"415895":31.5229710741,"416077":39.4408790499,"416293":47.2903677362,"416930":55.1975877328,"417115":63.1133510396,"417251":71.0959317975,"417298":78.8072213826},"duration":{"414822":0.0,"415179":2.2363888889,"415503":2.9830555556,"415631":3.3591666667,"415895":3.8255555556,"416077":4.1219444444,"416293":4.7058333333,"416930":6.5613888889,"417115":6.8327777778,"417251":7.0902777778,"417298":7.1933333333}}')
d = describe(t1) 
assert d.milage==78.8
assert d.date=='2019-09-12'
assert d.category=='Грузовой\\Грузопассажирский '
assert d.car=='a8185e3b-82a6-42a6-8673-4b3ca8349b7b'

In [0]:
"""Функции для уменьшения количества точек в треке"""

from vectors import growing_index, percentile_index, qs

def equidist(t, step_km):
  """Взять равные промежутки по *step_km* расстояния."""
  ix = growing_index(t.milage, step_km)
  return t.iloc[ix,:]

def equitime(t, step_min=5):
  """Взять равные промежутки по *step_min* минут времени."""
  time = step_min/60
  ix = growing_index(t.duration, time)
  return t.iloc[ix,:]

def spaced_dist(t, n: int, start=False, end=False):
  """Взять *n* равных промежутков по расстоянию.
  Опционально исключить начало и конец трека.
  """
  ix = percentile_index(t.milage, qs(n, start, end))
  return t.iloc[ix,:]

def spaced_time(t, n: int, start=False, end=False):
  """Взять *n* равных промежутков по времени.
  Опционально исключить начало и конец трека.
  """
  ix = percentile_index(t.duration, qs(n, start, end))
  return t.iloc[ix,:]

In [0]:
"""Графические функции для отрисовки треков"""

import matplotlib.pyplot as plt
from matplotlib import cm 

def get_color(i, n):
  if n<=10:
    cmap = cm.get_cmap(name='tab10')
    return cmap(i)
  else:
    cmap = cm.get_cmap(name='jet')
    return cmap(i/n)

def yield_trip_and_color(trips):
  n = len(trips)
  for i, t in enumerate(trips):
    col = get_color(i, n) 
    yield t, col

def make_plotter(axis_plotter_func):
  def plotter(trips, ax=None, title=''):
    if ax is None:
      f, ax = plt.subplots()
    n = len(trips)
    for i, t in enumerate(trips):
      col = get_color(i, n)
      axis_plotter_func(ax, t, col)
    ax.set_title(title)   
    return ax
  return plotter

def scatter(ax, t, col):
    ax.scatter(x=t.lon, y=t.lat, s=0.5, alpha=0.8, color=col)

def segments(ax, t, col):
    ax.plot(t.lon, t.lat, 
            lw=1, alpha=0.8, 
            linestyle=':', 
            color=col)  

def two_ends(ax, t, col):
    (x1, y1), (x2, y2) = ends(t)
    args = dict(s=60, edgecolors='white', alpha=0.8)
    ax.scatter(x=x1, y=y1, 
               marker='s', 
               color=col, 
               **args) 
    visual_offset = 0.005
    ax.scatter(x=x2+visual_offset, y=y2+visual_offset,
               marker='o', 
               color=col, 
               **args) 

def draw_drift(ax, t, col):
  (x1, y1), (x2, y2) = ends(t)
  plt.plot([x1, x2], [y1, y2], marker=None, lw=0.5, color=col)

plot_points      = make_plotter(scatter)
plot_connections = make_plotter(segments)
plot_bases       = make_plotter(two_ends)
plot_drift       = make_plotter(draw_drift)

def plot_trips(trips, title='', ax=None):
  ax = plot_points(trips, ax)
  plot_connections(trips, ax)
  plot_bases(trips, ax, title)

def plot_heatmap(mat, add_values=False, figsize=(9, 6)):
  import seaborn as sns
  f, ax = plt.subplots(figsize=figsize)
  sns.heatmap(mat,  
              vmin=0,  
              cmap="pink",
              annot=add_values,
              cbar=True,
              ax=ax)
  ax.invert_yaxis()
  plt.title("Расстояния между точками треков, км")
  plt.show()

# Полезно для анализа циклических треков:
# расстояния между точками трека "сам на себя"
#
# plot_heatmap(get_distances(t1, t1))


### Формирование выборки треков (класс Sample)

*Выборка треков*: все треки, которые мы анализируем на предмет сходимости, например, все поездки автобусов за неделю. 

Мы всегда сравниваем суточные треки ('машино-дни'). 

В максимальной группе сравнения 3724 'машино-дней', по автобусам и легковым - 1369 'машино-дней'. В используемом нами примере 49 треков ('машино-дней').

Переменные:

- `df_full: pd.DataFrame` - исходные данные треков (каждый трек содержит несколько тысяч точек) 
- `raw_trips: [pd.DataFrame]` - список из выбранных треков
- `trips: [pd.DataFrame]` - список из тех же выбранных треков, трек каждой поездки упрощен (небольшое количество точек - десятки или сотни).
К исходным данным добавлены продолжительность поездки и пробег от начала поездки (в каждой точке).


In [0]:
"""Приведение трека в стандартный вид и выборка треков."""

from dataclasses import dataclass, field
import pandas as pd

pd.set_option('mode.chained_assignment',None)

def as_list(xs):
  return xs if isinstance(xs, list) else [xs]

def subset(df_full, date_strings: [str], car_types: [str]) -> pd.DataFrame:
  """Выборка треков из *df_full*"""
  car_types = as_list(car_types)
  dates = [pd.Timestamp(dt).date() for dt in as_list(date_strings)]
  ix = df_full.date.isin(dates)
  ix = ix & (df_full.type.isin(car_types))
  return df_full[ix].sort_values(['date', 'car', 'time'])

def make_trip_list(selection: pd.DataFrame) -> [pd.DataFrame]:  
  trips = []
  for date in selection.date.unique():
   for car in selection.car.unique():
    ix = (selection.car == car) & (selection.date == date)
    trip = selection[ix]
    trips.append(trip)
  return [strip(t) for t in trips]

def strip(t: pd.DataFrame) -> pd.DataFrame:
  """Приведение трека в стандартный вид"""
  res = t[['time', 'lon', 'lat', 'date', 'car']]
  res['time'] = t.time.apply(lambda x: x.time())
  res['milage'] = t.dist.cumsum()
  res['duration'] = t.time_delta.cumsum() / (60*60)
  return res

@dataclass
class Sample:
  raw: [pd.DataFrame]
  reduced: [pd.DataFrame] = field(default_factory=list)
  
  def __len__(self) -> int:
    return len(self.raw)  

  def random_index(self) -> int:
    from random import randint 
    return randint(0, len(self))

  def combinations(self):
    from itertools import combinations
    N = len(self)
    return combinations(range(N),2)

def make_sample(df, func=None):
  raw_trips = make_trip_list(df)
  sample = Sample(raw_trips)
  if func is None: 
    return sample
  else:   
    return reduce_with(sample, func)  

def reduce_with(sample, func):
  raw = sample.raw
  reduced = [func(t) for t in raw] 
  return Sample(raw, reduced)


In [0]:
# Создаем выборку треков по дням и типам машин
days = ['2019-09-12']
modes = ['freight']
df_ = subset(df_full, days, modes)
sample = make_sample(df_)

# Упрощаем треки 
spacing_10 = lambda t: spaced_dist(t, n=10, start=True, end=True)
sample = reduce_with(sample, spacing_10)

In [0]:
# Нарисуем что получилось
trips = sample.reduced
raw_trips = sample.raw

ax = plot_points(trips)
msg = f'Выборка {modes} {days}, {len(trips)} поездок'
plot_connections(trips, ax, title=msg)
plt.show()

ax = plot_bases(trips)
plot_drift(trips, ax, title='Начало и конец треков')
plt.show()

In [0]:
# """Экспериментальная отрисовка 3D"""
trips = sample.reduced
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i,t in enumerate(trips):
  ax.plot(t.lon, t.lat, zs=i)  

### Отрисовка пары треков из выборки 

Предположим мы выбрали группу треков и упростили их. У нас есть пара треков, `t1` и `t2`. Покажем эти два трека на рисунках.

In [0]:
def plot_raw_and_reduced(r1, r2, t1, t2, title="Общий заголовок"):
  fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, 
                                 figsize=(10,4), 
                                 sharex=True, sharey=True) 
  plot_trips([r1, r2], ax=ax1, title="Исходные треки")
  plot_trips([t1, t2], ax=ax2, title="Упрощенные треки")
  plt.suptitle(title)
  plt.show()

def plot_two(sample: Sample, i: int, j: int):
  """Отрисовка пары маршрутов по индексам в выборке."""
  r1, r2 = sample.raw[i], sample.raw[j]
  t1, t2 = sample.reduced[i], sample.reduced[j] 
  title = f"Поездки {i} и {j}"
  plot_raw_and_reduced(r1, r2, t1, t2, title)  

a, b = 17, 35
t1 = trips[a]
t2 = trips[b]
r1 = raw_trips[a]
r2 = raw_trips[b]
assert describe(t1) == describe(r1)

print(describe(r1))
plot_raw_and_reduced(r1, r2, t1, t2)

### Пример: случайная пара треков из выборки

Ячейку ниже можно позапускать визуализации для случайных пар треков из набора.   

In [0]:
i = sample.random_index()
j = sample.random_index()
print(i, j)
plot_two(sample, i, j)

# Не пересекаются:
# 33 25
# 21 31 - но едут рядом

# Пересекаются:
# 27 29
# 29 19
# 22 0
# 26 22

# Очень близкие:
# 38 24
# 24 42

# Пересекаются, но лучше не объединять
# (разный характер поездок)
# 16 46

# По факту движения не пересекаются:
# 31 0

### Расчет близости пары маршрутов (класс Proximity)

Метрики близости для пары треков:
 - максимальное расстояние между точками маршрута
 - минимальное расстояние между точками маршрута
 - количество близких точек
 - покрытие одного маршрута другим

In [0]:
"""Расчет близости маршрутов."""

import numpy as np 
from scipy.spatial.distance import cdist

def distance_(a, b):
  x1, y1 = a
  x2, y2 = b
  if np.isnan(x1) or np.isnan(x2):
    return np.nan  
  else:    
    return distance_km(a, b)

def get_distances(t1, t2):
  p1 = points(t1)
  p2 = points(t2)
  return cdist(p1, p2, distance_)

def mask_diagonal(mat, offset: int):
  n = mat.shape[0]
  ix = np.triu_indices(n, offset)
  mat[ix] = np.nan
  ix = np.tril_indices(n, -offset)
  mat[ix] = np.nan
  return mat

def side_min(mat, axis):
  x = np.nanmin(mat, axis)
  return x[~np.isnan(x)]

def side_points_covered(mat, axis, radius_km):
    """Доля столбцов или строк матрицы расстояний между треками,
       в которых есть значения менее *radius_km*.
       Используется как показатель "перекрытия" треков одного маршрута
       другим маршрутом.
       axis принимает значени 0 или 1.
    """
    b = side_min(mat, axis)
    return len(b[b <= radius_km]) / len(b)

def side_points_covered_left(mat, radius_km):
  return side_points_covered(mat, 1, radius_km)

def side_points_covered_right(mat, radius_km):
  return side_points_covered(mat, 0, radius_km)

def proximity(sample: Sample, i: int, j: int):
  """Возвращает класс для анализа дистанций между парой треков."""
  return Proximity(sample.reduced[i], sample.reduced[j])

def pct(x: float) -> str:
  return f'{x*100:.0f}%'

class Proximity:
  """Расчет и анализ дистанций между точками двух треков t1 и t2."""
  def __init__(self, t1, t2):
    self.t1 = t1
    self.t2 = t2
    self.mat = get_distances(t1, t2)

  def size(self):
    return len(self.t1), len(self.t2)

  def total_cells_count(self):
    a, b = self.size()
    return a * b  

  def min(self):
    """
    Минимиальное расстояние между всеми точками двух треков.
    """
    return np.nanmin(self.mat)

  def max(self):
    """
    Максимальное расстояние между всеми точками двух треков.
    """
    return np.amax(self.mat)
  
  def points_covered(self, radius_km):
    """Кортеж с показателями сближения для треков t1 и t2.""" 
    return (side_points_covered_left(self.mat, radius_km), 
            side_points_covered_right(self.mat, radius_km)
            )

  def describe(self):
    return describe(self.t1), describe(self.t2)          

  def plot_heatmap(self):
    plot_heatmap(self.mat)

  def plot_trips(self):
    plot_trips([self.t1, self.t2])

# Полный пример поиска сходимости  (end-to-end) с параметрами






На входе:

- сырые данные треков поездок `df_full: pd.DataFrame`

Шаги:

1. создаем выборку треков по днями поездок и типам автомобилей
2. проводим грубую апроксимацию треков (например, 10 точками на трек)
3. отсекаем те пары треков, которые, скорее всего, не пересекаются
4. проводим более тонкую
апроксимацию оставшихся треков (например, с шагом каждый 1 или 5 км)
5. снова перебираем оставшиеся пары треков, в каждой паре считаем какую часть пробега одного трека покрывает другой трек, с заданным радиусом
6. треки в паре перекрываются не полностью, доля трека, которая совпадает в паре, разная для кажого трека:

```
                   3 км
12 км [------------****]           3/12 = 25%
                  [****----] 6 км   3/6 = 50%
```

7. сумма покрытий для пары треков представляет собой индикатор сходимости со значением от 0 до 2.0 (`0.75` для примера выше)
8. у нас есть пары треков, проранжированные по суммарному индикатору сходимости 

Результат:

- `prox_df: pd.DataFrame` пары треков с наибольшей сходимостью

На основе этих пар мы далее показываем каталог близких поездок с высокими индикатором сходимости, как хорошие, так и пограничные случаи.  



In [0]:
# 1. Создаем выборку треков по дням и типам машин и упрощаем ее 
days = ['2019-09-12']
modes = ['freight']
df_subset = subset(df_full, days, modes)

# Создаем упрощенные треки 
ten_segments = lambda t: spaced_dist(t, n=10, start=True, end=True)
sample1 = reduce_with(make_sample(df_), ten_segments)

In [0]:
# 2. Определяем пары треков - кандидаты на сближение
close_enough_km = 5 # На каком расстоянии считать приближенные 
                    # треки сблизившимися?

def df_rows_short(sample):
  for (i, j) in sample.combinations():
      p = proximity(sample, i, j)
      yield (i, j), p.min(), p.max()

def pair_df(sample):
  return pd.DataFrame(df_rows_short(sample),
                      columns = ['pair', 'min_dist', 'max_dist'])

df = pair_df(sample1)
pairs = df[df.min_dist < close_enough_km].pair
print("Мы уменьшили количество пар для перебора примерно вдвое:")
len(pairs), len(df)

In [0]:
zoom = False # True False

df.plot.scatter(x='min_dist', y='max_dist', alpha=0.75)
plt.title("Минимальные и максимальные дистанции по парам треков")
plt.axvline(x=5, c='red')
if zoom:
  plt.xlim(0,10)
  plt.ylim(0,150)
plt.show()

In [0]:
# 3. Строим треки с увеличенной точностью
step_km = 2.5  # шаг упрощения трека, предлагаем использовать 1, 2.5 или 5 км
every_km = lambda t: equidist(t, step_km)
sample2 = reduce_with(sample1, every_km)

In [0]:
from pandas.testing import assert_frame_equal 
assert_frame_equal(sample1.raw[19], sample2.raw[19])
assert sample2.raw[19].iloc[0].milage == 0 

In [0]:
# 4. Определяем попарно, какую долю трека покрывает другой трек
from tqdm import tqdm

proximity_radius = 5 # На каком расстоянии мы считаем точки двух треков близкими?
                     # Рекомендуется от 0.5 до 5 км

def df_rows_(sample, pairs, radius):
  for (i, j) in pairs:
      p = proximity(sample, i, j)
      a, b = p.points_covered(radius)
      yield dict(pair=(i, j), 
                 min_dist=p.min(), 
                 max_dist=p.max(),
                 cov_t1=a, 
                 cov_t2=b,
                 len_t1=length(p.t1),
                 len_t2=length(p.t2)
                 )

def proximity_df(sample, pairs, radius=proximity_radius):
  gen = df_rows_(sample, pairs, radius)
  return (pd.DataFrame(gen)
            .assign(cov_total=lambda df: df.cov_t1+df.cov_t2)
            .sort_values('cov_total', ascending=False)
            .reset_index(drop=True))

# Результат расчетов - метриками близости(сходимости) для каждой пары треков
prox_df = proximity_df(sample2, pairs, proximity_radius)


In [0]:
prox_df.head(20)

In [0]:
plt.scatter(prox_df.index, prox_df.cov_total, s=0.5)
plt.title("Суммарный индикатор сходимости\nдля пары треков (от 0 до 2.0)")
plt.show()

plt.scatter(prox_df.cov_t1, prox_df.cov_t2, s=0.8, alpha=0.8)
plt.title("Покрытие маршрутов в паре треков")
plt.show()

prox_df['len_total'] = prox_df.len_t1 + prox_df.len_t2 
plt.scatter(prox_df.len_total, prox_df.cov_total, s=0.8, alpha=0.8)
plt.title("Суммарная длина и сходимость пары треков")
plt.show()

## Все в одной функции 


In [0]:
def search_proximity(sample: Sample, 
           approx_func1 = lambda t: spaced_dist(t, n=10, start=True, end=True),
           approx_func2 = lambda t: equidist(t, step_km = 2.5), 
           search_radius1 = 5,
           search_radius2 = 2.5*1.2) -> pd.DataFrame:
  """
  Функция для поиска сходимости.
  
  Принимает аргумент *sample:* - набор треков типа *Sample*.

  Дефолтные параметры для поиска сходмости.

  Апроксимации треков:
  approx_func1 :: первичная апроксимация - трек представлен как 10 равных 
                  по длине сегментов
  approx_func2 :: улучшенная апроксимация трека - каждые сегменты с равной 
                  длиной 2.5 км 

  Поиск сходимости: 
  search_radius1 :: первый поиск сходимости - в радиусе 5 км (рекомендуется 
                    значение от 3 до 10 км)
  search_radius2 :: второй поиск сходимости - в радиусе 2.5 км + 20%

  Возвращет датафрейм с параметрами сходимости.
  """
  print(f'Working with sample of {len(sample)} tracks')
  # делаем грубую апроксимацию треков
  sample = reduce_with(sample, approx_func1)
  print('Simplified tracks')
  comb = len(list(sample.combinations()))
  print(comb, 'pairs are possible')
  # получаем список пар близких треков
  df = pair_df(sample)
  pairs = df[df.min_dist < search_radius1].pair
  print(f'Found {len(pairs)} pairs to calculate proximity')
  # делаем улучшенную апроксимацию треков 
  sample = reduce_with(sample, approx_func2)
  print('Made better approximation of tracks')
  print('Searching for proximity (can take long time, wait)...')
  # получаем итоговую информацию о списке сошедшихся пар треков
  prox_df = proximity_df(sample, pairs, search_radius2)
  print('Found close track pairs, reporting...')
  return prox_df

days = ['2019-09-12']
modes = ['freight']
df_subset = subset(df_full, days, modes)
sample = make_sample(df_subset) 
prox_df = search_proximity(sample)

## Отрисовка пар сходимости

In [0]:
def make_circle_plot(list_of_cars, list_of_pairs, figsize=(5, 5), dpi=150):
  import math

  list_of_cars = np.array(list_of_cars)

  r = 10

  x = []
  y = []

  fig = plt.figure(figsize=figsize, dpi=dpi)

  n = len(list_of_cars)
  for i in range(n):
      
      angle = (i/n)*(math.pi*2)
      x.append(r*math.cos(angle))
      y.append(r*math.sin(angle))

  plt.scatter(x,y, s=100, marker='o', alpha=0.7, c='orange')

  n = len(list_of_pairs)
  for k in range(n):
      pair = list_of_pairs[k]
      i = np.where(list_of_cars == pair[0])[0][0]
      j = np.where(list_of_cars == pair[1])[0][0]
      
      plt.plot([x[i],x[j]],[y[i],y[j]], 'o-', c='orange', linewidth=3, alpha=0.7, markersize=10)

  plt.axes().set_aspect('equal', 'datalim')
  plt.show()

def all_ride_ids(prox_df):
  return list(set(prox_df.pair.sum()))

list_of_cars = all_ride_ids(prox_df)
list_of_pairs = prox_df[prox_df.cov_total > 1.7].pair.tolist()

make_circle_plot(list_of_cars, list_of_pairs, figsize=(5, 5), dpi=100)  

In [0]:
# refactoring from above
import matplotlib.pyplot as plt
import numpy as np 
import math

def dot_positions(n: int, radius: float):
  xs = []
  ys = []  
  for i in range(n):
    angle = math.pi/2 - (i/n)*(math.pi*2)
    x=radius*math.cos(angle)
    xs.append(x)
    y=radius*math.sin(angle)
    ys.append(y)
  return xs, ys   
 
def draw_ciferblat(xs, ys):   
  args = dict(s=100, marker='o', alpha=0.7, c='orange')
  ax = plt.scatter(xs, ys, **args)
  plt.axes().set_aspect('equal', 'datalim')
  plt.axes().axis('off')
  return ax

def draw_line(x1, x2, y1, y2):
  plt.plot([x1, x2],[y1, y2], 'o-', c='orange', linewidth=3, alpha=0.7, markersize=10) 

def count_rides(pairs):
  ps = [p for pair in pairs for p in pair]
  return len(set(ps))

def ordered_dict(pairs):
  ps = [p for pair in pairs for p in pair]
  c = Counter(ps)
  return {z:i for i,z in enumerate([x[0] for x in c.most_common()])}

def plot_clockwise(pairs, radius=5, figsize=(5, 5), dpi=150):
  plt.figure(figsize=figsize, dpi=dpi)
  n = count_rides(pairs)
  xs, ys = dot_positions(n, radius)
  mapper = ordered_dict(pairs)
  draw_ciferblat(xs, ys)
  for pair in pairs:
    i, j = pair
    i = mapper[i]
    j = mapper[j]
    draw_line(xs[i], xs[j], ys[i], ys[j])
  return n 

threshold = 1.7
pairs = prox_df.query(f"cov_total>{threshold}").pair.to_list()
n = plot_clockwise(pairs)
plt.title(f'Поездок со сходимостью в паре выше {threshold}: {n}') 
plt.show() 



# 5. Каталог сходимости (примеры поездок)

Используемые данные:

- `prox_df: pd.DataFrame` пары треков с наибольшей сходимостью
- выборка данных `sample2: Sample`

На основе этих пар мы далее показываем каталог близких поездок с высокими индикатором сходимости, как хорошие, так и пограничные случаи.


In [0]:
def dec(x: float) -> str:
  return f"{x:.2f}"

def info(sample, i, j, search_radius, heatmap=True):
  """
  Просмотр результатов расчета близости точек двух треков 
  с заданным радиусом сближения *search_radius*. 
  
  Точки ближе  чем *search_radius* километров друг от друга 
  считаются сошедшимися.     
  """
  print("Треки", i, j)  
  print(f"Радиус поиска", search_radius)
  p = proximity(sample, i, j)
  d1, d2 = p.describe()
  a, b = p.points_covered(search_radius)
  print(f"Суммарная сходимость", dec(a+b), "(максимум 2)")
  for d, cov in zip([d1, d2], [a, b]):
    print()
    print("Автомобиль", d.car)
    print("Тип", d.category)
    print("Дата", d.date)
    print("Пробег", d.milage)
    print("Перекрытие другим треком", dec(cov))
  plot_two(sample, i, j)
  if heatmap:
    p.plot_heatmap()

In [0]:
info(sample2, 24, 39, 2.5)

In [0]:
# неравномерное перекрытие - пример
info(sample2, 21, 34, 2.5)

## Top 10 наиболее близких поездок

Ниже мы показываем 10 пар поездок с коэффициентами сходимости близкими к 2. В этих парах поездок каждый путь покрывается другим путем почти на 100%.

Возможны альтернативные сортировки, например:
- поездки, где уменьшение дублирования даст наибольшее сниение пробега (высокая сходимость и длинный пробег);
- поездки похожих типов (близкая длина и локация пути);
- поездки с одной парковки.


In [0]:
new_radius = 1 # посмотрим на сближение с еще более точным радиусом поиска
n = 10 
top10 = [(i, j, new_radius) for [i, j] in prox_df.head(n).pair.to_list()]

for (i, j, r) in top10:
  info(sample2, i, j, r, heatmap=True)

## Примеры непересекающихся треков

In [0]:
# Строго не пересекаются:
# 33 25
info(sample2, 33, 25, 2.5, False)

In [0]:
# Неравномерное перекрытие - межгород и город
info(sample2, 21, 34, 2.5, False)

In [0]:
# Едут рядом, не пересекаются, но если задать больший 
# радиус перекрытия (10 вместо 2.5), 
# то часть пути будет считаться близкой
info(sample2, 21, 31, 2.5, False)

In [0]:
# Пересекаются в проекциях, но по факту движения не пересекаются:
# 31 0
info(sample2, 31, 0, 2.5)

# Результаты

1. Мы построили работающий прототип системы для выявления близких треков поездок, реализованный в виде ноутбука Jupiter/Google Colab.

2. За счет апрокисмации треков небольшим количеством точек (от 10 до 200 вместо нескольких тысяч) мы существенно снижаем время расчетов.

3. В группу для сравнения треков может быть включена произвольная выборка "машинодней" (поездок автомобиля в течение суток) - по дням, типам машин или другим признакам.

4. На примере одного дня грузовых перевозок 12.09.2019 мы находим значительное количество близких пар  треков. 

5. В паре треков сходимость (пересечение) измеряется долей точек трека одного автомобиля, которые находятся не далее заданного расстояния от любых точек трека другого автомобиля.

6. В "каталоге поездок" мы показываем на пары треков с высокими показателями пересечения. Эти иллюстрации могут использоваться
для обсуждения конкретных ситуаций с владельцами парка. 


# Экспериментальные расчеты 

### Cортировка по времени

- упрощаем трек равными временными промежутками внтури суток (например, 1 час, 30 минут, 10 минут)
- рассматриваем только основную диагональ матрицы расстояний с допуском на сдвиг по времени плюс или минус n часов
- может сбоить на коротких отрисовках (1 минута)

In [0]:
import datetime as dt

from datetime import time, timedelta, date, datetime

def time_seconds(t: Trip) -> [int]:
  return [to_seconds(x) for x in t.time]

def to_seconds(dt_obj: dt.time) -> int:
  return (dt_obj.hour*60 + dt_obj.minute)*60 + dt_obj.second

def to_str(seconds: int) -> str:
  return str(timedelta(seconds=seconds))

def to_time(seconds: int) -> dt.time:
  return (datetime.min + timedelta(seconds=seconds)).time()

def yield_times(increment) -> [dt.time]:
    """
    Предоставить метки времени внутри суток с равными промежутками.
    """
    delta = pd.Timedelta(increment)
    end_of_day = pd.Timestamp('23:59:59') 
    x = pd.Timestamp('00:00:00')
    yield x.time()
    while True:        
        x = x + delta
        if x > end_of_day:
          break
        yield x.time()
    yield end_of_day.time()   

def time_range(increment="1h") -> [dt.time]:
  return [to_seconds(x) for x in yield_times(increment)]

def match(xs: [int], 
          marks: [int], 
          tol: int):
  marks = iter(marks)
  for m in marks:
    if m < xs[0]:
      yield m # x was not available
    else:
      break  
  for x in xs:
    if x < m:
      continue
    if (x-m < tol):
      yield x
    m = next(marks)
  for m in marks:
      yield m # x was not available

def trim_in_time(t: Trip, increment='1h', tolerance='5min') -> Trip:
  """
  Сократить трек до точек по времени внутри суток.
  Частота по умолчанию - каждый целый час. 
  """
  matches = match(xs=time_seconds(t),
                  marks=time_range(increment),
                  tol= pd.Timedelta(tolerance).seconds)
  matches = [to_time(m) for m in matches]
  return pd.DataFrame(matches, columns=['time']).merge(t, on='time', how='left') 

In [0]:
# Посмотрим два близких по географии трека `t1` (19) и `t2` (24)
# Они сближались по фактическому времени поездки

t1 = sample1.raw[19]
t2 = sample1.raw[24]
reducer = lambda t: trim_in_time(t, '10min', '5min')
p = Proximity(reducer(t2), reducer(t1))
p.mat = mask_diagonal(p.mat, offset=3)
p.plot_trips()
p.plot_heatmap()
p.points_covered(1)